In [23]:
from dash import Dash, html, dcc, Input, Output
import pandas as pd
import plotly.express as px
import geopandas as gpd
import shapely 
import numpy as np
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
import dash
from dash_bootstrap_templates import load_figure_template
import plotly.graph_objects as go
import dash_daq as daq
# import sys
# from pathlib import Path
# sys.path.append(str(Path(__file__).parent))
from figure_functions import barchart_heating_systems, boxplot_co2, boxplot_investment
from constants import INJECTION_PRICE, GAS_PRICE, ELECTRICITY_PRICE, ELEC_CO2_INTENSITY, ELEC_CO2_INTENSITY_ELECTRIFICATION, COLS
from tab_layout import make_layout_scenario_maker, make_sliders
from server import app
from functions import map_epc
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

from luminalysis_functions import example_cases, dynamic_co2, fit_co2, mod_solar

In [24]:
# df = pd.read_feather('data/archetypes_luminus_trimmed.ftr')
# def hs(scenario_name, gas, elec, oil, heating_consumption):
#     if scenario_name == 'current situation':
#         return 'Oil' if oil >= heating_consumption else 'Gasboiler'
#     elif 'hp' in scenario_name.lower() or 'warmtepomp' in scenario_name.lower():
#         return 'Heat Pump'
#     elif 'gasboiler' in scenario_name.lower():
#         return 'Gasboiler'
#     else:
#         return 'Gasboiler'

# def dhw_system(scenario_name, gas, elec, heating_system):
#     if 'heat pump boiler' in scenario_name.lower():
#         return 'HP Boiler'
#     elif heating_system == 'Heat Pump':
#         if gas == 0:
#             return 'HP Boiler'
#         else:
#             return 'Gasboiler'
#     else:
#         return 'Gasboiler'
# df['heating_system'] = df.apply(lambda x: hs(x.scenario_name,
#                                              x.gas_consumption,
#                                              x.electricity_consumption,
#                                              x.oil_consumption,
#                                              x.heating_consumption), axis = 1)
# df['dhw_system'] = df.apply(lambda x: dhw_system(x.scenario_name,
#                                              x.gas_consumption,
#                                              x.electricity_consumption,
#                                              x.heating_system), axis = 1)

data read-in

In [25]:


#df = pd.read_csv('data/archetypes_luminus.csv', usecols = COLS)
df = pd.read_feather('data/archetypes_luminus_trimmed.ftr')
df['tco'] = df.total_investment_cost + df.electricity_consumption*15*ELECTRICITY_PRICE['BE'] + df.gas_consumption*GAS_PRICE['BE'] * 15+ df.total_oil_cost -df.electricity_injected * INJECTION_PRICE['BE'] * 15
df['total_co2'] = df.electricity_consumption * 0.171 + df.gas_consumption * 0.056 * 3.6 + df.oil_consumption * 0.074 * 3.6 - df.electricity_injected * 0.171
df.loc[ (df['heating_system'] == 'Heat Pump')& (df['dhw_system'] == 'HP Boiler') & (df.oil_consumption == df.domestic_hot_water_consumption), 'electricity_consumption'] = df.electricity_consumption + df.oil_consumption
df.loc[ (df['heating_system'] == 'Heat Pump')& (df['dhw_system'] == 'HP Boiler') & (df.oil_consumption == df.domestic_hot_water_consumption), 'oil_consumption'] = 0
# df['mapped_epc'] = df.apply(lambda x: map_epc( epc = x.epc_ind, 
#                                                 co2 = x.total_co2 / x.total_floor_area), axis = 1)
optimal_unmodded_temp = df[df.epc_ind<=100].sort_values(by = 'tco', ascending = True).drop_duplicates(subset = 'name', keep = 'first')
optimal_unmodded_temp['dataset'] = 'unmodded'
optimal_modded_temp = df[df.epc_ind<=100].sort_values(by = 'tco', ascending = True).drop_duplicates(subset = 'name', keep = 'first')
optimal_modded_temp['dataset'] = 'modded'



In [26]:
testcase2 = df[(df['epc_ind'] <= 100) & (df['heating_system'] == 'Heat Pump')& (df['dhw_system'] == 'HP Boiler')].sort_values(by='tco', ascending=True).drop_duplicates(subset=['name'], keep='first')

In [27]:
testcase2[testcase2.oil_consumption > 0][['heating_consumption', 'domestic_hot_water_consumption', 'electricity_consumption', 'gas_consumption', 'oil_consumption', 'scenario_name']]

,heating_consumption,domestic_hot_water_consumption,electricity_consumption,gas_consumption,oil_consumption,scenario_name


fitting 15 year co2 emissions

In [28]:
x_fit, y_fit_central, y_fit_electrification, fig = fit_co2(central_scenario = ELEC_CO2_INTENSITY, electrification_scenario = ELEC_CO2_INTENSITY_ELECTRIFICATION, returnplot = True)
# Plot the data and the fit
fig.show()

In [29]:
# # Convert dictionary to arrays
# years = np.array(list(ELEC_CO2_INTENSITY.keys()))
# central_scenario = np.array(list(ELEC_CO2_INTENSITY.values()))
# electrification_scenario = np.array(list(ELEC_CO2_INTENSITY_ELECTRIFICATION.values()))

# # Exponential decay function
# def exp_decay(x, a, b):
#     return a * np.exp(b * x)

# # Fit the central scenario data
# popt_central, pcov_central = curve_fit(exp_decay, years - 2020, central_scenario, p0=(0.171, -0.1))
# a_central, b_central = popt_central

# # Fit the electrification scenario data
# popt_electrification, pcov_electrification = curve_fit(exp_decay, years - 2020, electrification_scenario, p0=(0.171, -0.1))
# a_electrification, b_electrification = popt_electrification

# # Generate data for plotting
# x_fit = np.linspace(2020, 2040, 21)
# y_fit_central = exp_decay(x_fit - 2020, *popt_central)
# y_fit_electrification = exp_decay(x_fit - 2020, *popt_electrification)
# Plot the data and the fit
# fig = plt.figure(figsize=(10, 6))
# plt.plot(years, central_scenario, 'o', label='Central Scenario Data')
# plt.plot(x_fit, y_fit_central, '-', label=f'Central Scenario Fit: a={a_central:.3f}, b={b_central:.3f}')
# plt.plot(years, electrification_scenario, 'o', label='Electrification Scenario Data')
# plt.plot(x_fit, y_fit_electrification, '-', label=f'Electrification Scenario Fit: a={a_electrification:.3f}, b={b_electrification:.3f}')
# plt.xlabel('Year')
# plt.ylabel('CO2 Intensity (kg/kWh)')
# plt.title('CO2 Intensity Projections until 2040')
# plt.legend()
# plt.grid(True)
# plt.show()


In [30]:
#we modify solar panels, and we also calculate yearly co2 emissions based on the co2 intensity of electricity for each year
df = dynamic_co2(df, x_fit, y_fit_central, start_year = 2025, solar = True)
df = mod_solar(df, co2_flag=True)

years = np.arange(2025, 2025 + len(y_fit_central[5:]))
for year in range(len(years)):
    df[f'solar_injection_emissions_{year + 2025}'] = (
        df[f'modified_emissions_{year + 2025}'] - 
        df[f'emissions_{year + 2025}'] 
    )

In [31]:
for x in df.columns:
    print(x)

Unnamed: 0
epc_ind
epc_label
total_co2
scenario_name
gas_consumption
oil_consumption
total_floor_area
heating_consumption
electricity_injected
total_investment_cost
electricity_consumption
domestic_hot_water_consumption
total_subsidies
total_oil_cost
pv_production
name
building_type
construction_year
scale_kadaster
heating_system
tco
dhw_system
emissions_2025
fossil_emissions_2025
electricity_emissions_2025
modified_emissions_2025
emissions_2026
fossil_emissions_2026
electricity_emissions_2026
modified_emissions_2026
emissions_2027
fossil_emissions_2027
electricity_emissions_2027
modified_emissions_2027
emissions_2028
fossil_emissions_2028
electricity_emissions_2028
modified_emissions_2028
emissions_2029
fossil_emissions_2029
electricity_emissions_2029
modified_emissions_2029
emissions_2030
fossil_emissions_2030
electricity_emissions_2030
modified_emissions_2030
emissions_2031
fossil_emissions_2031
electricity_emissions_2031
modified_emissions_2031
emissions_2032
fossil_emissions_2032


basic analyses

In [32]:
testcase = df[df.epc_ind<=100].sort_values(by = 'tco', ascending = True).drop_duplicates(subset = 'name', keep = 'first')
testcase = testcase[testcase.name == 'type_0']
fig = go.Figure()
for heating_system in testcase.heating_system.unique():
    for dhw_system in testcase.dhw_system.unique():
        tmp = testcase[(testcase.heating_system == heating_system) & (testcase.dhw_system == dhw_system)]
        years = np.arange(2025, 2025 + len(y_fit_central[5:]))
        total_co2_cumulative = tmp['total_co2'].values * np.array([1 for i in range(16)]).cumsum()
        modified_total_co2_cumulative = tmp['total_co2_modified'].values * np.array([1 for i in range(16)]).cumsum()
        new_emissions_cumulative = tmp[[f'emissions_{year + 2025}' for year in range(len(y_fit_central[5:]))]].cumsum(axis=1)
        modified_new_emissions_cumulative = tmp[[f'modified_emissions_{year + 2025}' for year in range(len(y_fit_central[5:]))]].cumsum(axis=1)
        #new_emissions_cumulative = tmp[[f'emissions_{year + 2025}' for year in range(len(y_fit_central[5:]))]]
        # Create the Plotly scatter plot


        # Add total_co2 cumulative emissions
        # fig.add_trace(go.Scatter(
        #     x=years,
        #     y=total_co2_cumulative,
        #     mode='lines+markers',
        #     name='Total CO2 (Original)'
        # ))
        fig.add_trace(go.Scatter(
            x=years,
            y=new_emissions_cumulative.values[0],
            mode='lines+markers',
            name=f'{heating_system} + {dhw_system}'
        ))
        fig.add_trace(go.Scatter(
            x=years,
            y=modified_new_emissions_cumulative.values[0],
            mode='lines+markers',
            name=f'modified {heating_system} + {dhw_system}'
        ))
        fig.add_trace(go.Scatter(
            x=years,
            y=total_co2_cumulative,
            mode='lines+markers',
            name=f'standard {heating_system} + {dhw_system}'
        ))
        fig.add_trace(go.Scatter(
            x=years,
            y=modified_total_co2_cumulative,
            mode='lines+markers',
            name=f'standard modified {heating_system} + {dhw_system}'
        ))
        
# # Add newly calculated cumulative emissions
# tmp = testcase[testcase.heating_system == 'Heat Pump']
# years = np.arange(2025, 2025 + len(y_fit_central[5:]))
# #total_co2_cumulative = tmp['total_co2'].values * np.array([1 for i in range(16)]).cumsum()
# new_emissions_cumulative = tmp[[f'emissions_{year + 2025}' for year in range(len(y_fit_central[5:]))]].cumsum(axis=1)
# for i in range(new_emissions_cumulative.shape[0]):
#     fig.add_trace(go.Scatter(
#         x=years,
#         y=new_emissions_cumulative.iloc[i],
#         mode='lines+markers',
#         name=f'{tmp.heating_system.values}'
#     ))

# Update layout
fig.update_layout(
    title='Yearly Cumulative Emissions',
    xaxis_title='Year',
    yaxis_title='Cumulative Emissions (kg CO2)',
    legend_title='Emission Categories'
)

# Show the plot
fig.show()

In [33]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# Filter the DataFrame
testcase2 = df[(df['epc_ind'] <= 100) & (df['heating_system'] == 'Gasboiler')].sort_values(by='tco', ascending=True).drop_duplicates(subset=['name'], keep='first')

# Extract the years from the columns
years = sorted([int(col.split('_')[-1]) for col in testcase2.columns if 'emissions_' in col])

# Initialize lists to store the data
data = []

# Collect data for each year and each building type
for year in years:
    for building_type, group in testcase2.groupby('building_type'):
        modified_emissions = group[f'modified_emissions_{year}'].sum()
        solar_emissions = group[f'solar_injection_emissions_{year}'].sum() * -1  # Make solar emissions negative
        net_emissions = group[f'emissions_{year}'].sum()
        # modified_emissions = group[f'modified_emissions_{year}'].mean()
        # solar_emissions = group[f'solar_injection_emissions_{year}'].mean() * -1  # Make solar emissions negative
        # net_emissions = group[f'emissions_{year}'].mean()
        data.append([year, building_type, 'Modified CO2 Emissions', modified_emissions])
        data.append([year, building_type, 'Solar CO2 Emissions', solar_emissions])
        data.append([year, building_type, 'Net Emissions', net_emissions])

# Create a DataFrame from the collected data
emissions_df = pd.DataFrame(data, columns=['Year', 'Building Type', 'Emissions Type', 'Emissions'])

# Get unique building types
building_types = emissions_df['Building Type'].unique()

# Create subplots
fig = make_subplots(rows=3, cols=len(building_types), shared_yaxes=True, subplot_titles=building_types)

# Add traces for each building type
for i, building_type in enumerate(building_types, start=1):
    subset = emissions_df[emissions_df['Building Type'] == building_type]

    fig.add_trace(go.Scatter(
        x=subset[subset['Emissions Type'] == 'Modified CO2 Emissions']['Year'],
        y=subset[subset['Emissions Type'] == 'Modified CO2 Emissions']['Emissions'],
        fill='tonexty',
        name=f'{building_type} - Modified CO2 Emissions'
    ), row=1, col=i)
    fig.add_trace(go.Scatter(
        x=subset[subset['Emissions Type'] == 'Solar CO2 Emissions']['Year'],
        y=subset[subset['Emissions Type'] == 'Solar CO2 Emissions']['Emissions'],
        fill='tonexty',
        name=f'{building_type} - Solar CO2 Emissions'
    ), row=1, col=i)
    fig.add_trace(go.Scatter(
        x=subset[subset['Emissions Type'] == 'Solar CO2 Emissions']['Year'],
        y=subset[subset['Emissions Type'] == 'Net Emissions']['Emissions'],
        fill='tonexty',
        name=f'{building_type} - Net CO2 Emissions'
    ), row=2, col=i)
    fig.add_trace(go.Histogram(x = testcase2[testcase2.building_type == building_type].total_investment_cost), row = 3, col = i)


# Update layout
fig.update_layout(
    title='CO2 Emissions Over Years by Building Type with gasboilers and A labels',
    xaxis_title='Year',
    yaxis_title='CO2 Emissions (kg)',
    showlegend=True,
    height = 800, width = 1600
)

# Update x-axes titles for each subplot
for i in range(1, len(building_types) + 1):
    fig.update_xaxes(title_text='Year', row=1, col=i)

# Update y-axis title only for the first subplot
fig.update_yaxes(title_text='Gross CO2 Emissions (kg)', row=1, col=1)
fig.update_yaxes(title_text='Net CO2 Emissions (kg)', row=2, col=1)
fig.update_yaxes(title_text='Count', row=3, col=1)
fig.update_yaxes(range = [-45000, 120000], row = 1)
fig.update_yaxes(range = [-20000, 100000], row = 2)
fig.update_yaxes(range = [0, 30], row = 3)
for i in range(len(building_types)+ 1):
    fig.update_xaxes(title_text='Investment cost [€]', row=3, col=i)

# Show the plot
fig.show()
fig2 = fig

In [34]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# Filter the DataFrame
testcase2 = df[(df['epc_ind'] <= 100) & (df['heating_system'] == 'Heat Pump')& (df['dhw_system'] == 'HP Boiler')].sort_values(by='tco', ascending=True).drop_duplicates(subset=['name'], keep='first')

# Extract the years from the columns
years = sorted([int(col.split('_')[-1]) for col in testcase2.columns if 'emissions_' in col])

# Initialize lists to store the data
data = []

# Collect data for each year and each building type
for year in years:
    for building_type, group in testcase2.groupby('building_type'):
        modified_emissions = group[f'modified_emissions_{year}'].sum()
        fossil_emissions = group[f'fossil_emissions_{year}'].sum()
        electricity_emissions = group[f'electricity_emissions_{year}'].sum()
        # electricity_emissions = (group[f'modified_emissions_{year}'] - group[f'gas_consumption'] *0.056 * 3.6- group[f'oil_consumption']*0.074*3.6).sum()
        # fossil_emissions = electricity_emissions = (group[f'modified_emissions_{year}'] - group['elecricity_emissions']).sum()
        solar_emissions = group[f'solar_injection_emissions_{year}'].sum() * -1  # Make solar emissions negative
        net_emissions = group[f'emissions_{year}'].sum()
        data.append([year, building_type, 'Modified CO2 Emissions', modified_emissions])
        data.append([year, building_type, 'Solar CO2 Emissions', solar_emissions])
        data.append([year, building_type, 'Net Emissions', net_emissions])
        data.append([year, building_type, 'Fossil Emissions', fossil_emissions])
        data.append([year, building_type, 'Electricity Emissions', electricity_emissions])

# Create a DataFrame from the collected data
emissions_df = pd.DataFrame(data, columns=['Year', 'Building Type', 'Emissions Type', 'Emissions'])

# Get unique building types
building_types = emissions_df['Building Type'].unique()

# Create subplots
fig = make_subplots(rows=3, cols=len(building_types), shared_yaxes=True, subplot_titles=building_types)

# Add traces for each building type
for i, building_type in enumerate(building_types, start=1):
    subset = emissions_df[emissions_df['Building Type'] == building_type]

    fig.add_trace(go.Scatter(
        x=subset[subset['Emissions Type'] == 'Modified CO2 Emissions']['Year'],
        y=subset[subset['Emissions Type'] == 'Modified CO2 Emissions']['Emissions'],
        fill='tonexty',
        stackgroup = '1',
        name=f'{building_type} - Modified CO2 Emissions'
    ), row=1, col=i)

    fig.add_trace(go.Scatter(
        x=subset[subset['Emissions Type'] == 'Solar CO2 Emissions']['Year'],
        y=subset[subset['Emissions Type'] == 'Solar CO2 Emissions']['Emissions'],
        fill='tonexty',
        stackgroup = '2',
        name=f'{building_type} - Solar CO2 Emissions'
    ), row=1, col=i)
    fig.add_trace(go.Scatter(
        x=subset[subset['Emissions Type'] == 'Solar CO2 Emissions']['Year'],
        y=subset[subset['Emissions Type'] == 'Net Emissions']['Emissions'],
        fill='tonexty',
        text = 'Net Emissions',
        stackgroup = '3',
        name=f'{building_type} - Net CO2 Emissions'
    ), row=2, col=i)
    
    fig.add_trace(go.Histogram(x = testcase2[testcase2.building_type == building_type].total_investment_cost), row = 3, col = i)


# Update layout
fig.update_layout(
    title='CO2 Emissions Over Years by Building Type with heat pumps and A labels',
    xaxis_title='Year',
    yaxis_title='CO2 Emissions (kg)',
    showlegend=True,
    height = 800, width = 1600
)

# Update x-axes titles for each subplot
for i in range(1, len(building_types) + 1):
    fig.update_xaxes(title_text='Year', row=1, col=i)

# Update y-axis title only for the first subplot
fig.update_yaxes(title_text='Gross CO2 Emissions (kg)', row=1, col=1)
fig.update_yaxes(title_text='Net CO2 Emissions (kg)', row=2, col=1)
fig.update_yaxes(title_text='Count', row=3, col=1)
fig.update_yaxes(range = [-45000, 120000], row = 1)
fig.update_yaxes(range = [-20000, 100000], row = 2)
fig.update_yaxes(range = [0, 30], row = 3)
for i in range(len(building_types)+ 1):
    fig.update_xaxes(title_text='Investment cost [€]', row=3, col=i)
# Show the plot
fig.show()

In [35]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# Filter the DataFrame
testcase2 = df[(df['epc_ind'] <= 100) & (df['heating_system'] == 'Heat Pump')& (df['dhw_system'] == 'HP Boiler')].sort_values(by='tco', ascending=True).drop_duplicates(subset=['name'], keep='first')

# Extract the years from the columns
years = sorted([int(col.split('_')[-1]) for col in testcase2.columns if 'emissions_' in col])

# Initialize lists to store the data
data = []

# Collect data for each year and each building type
for year in years:
    for building_type, group in testcase2.groupby('building_type'):
        modified_emissions = group[f'modified_emissions_{year}'].sum()
        fossil_emissions = group[f'fossil_emissions_{year}'].sum()
        electricity_emissions = group[f'electricity_emissions_{year}'].sum()
        solar_emissions = group[f'solar_injection_emissions_{year}'].sum() * -1  # Make solar emissions negative
        net_emissions = group[f'emissions_{year}'].sum()
        data.append([year, building_type, 'Modified CO2 Emissions', modified_emissions])
        data.append([year, building_type, 'Solar CO2 Emissions', solar_emissions])
        data.append([year, building_type, 'Net Emissions', net_emissions])
        data.append([year, building_type, 'Fossil Emissions', fossil_emissions])
        data.append([year, building_type, 'Electricity Emissions', electricity_emissions])

# Create a DataFrame from the collected data
emissions_df = pd.DataFrame(data, columns=['Year', 'Building Type', 'Emissions Type', 'Emissions'])

# Get unique building types
building_types = emissions_df['Building Type'].unique()

# Create subplots
fig = make_subplots(rows=3, cols=len(building_types), shared_yaxes=True, subplot_titles=building_types)

# Add traces for each building type
for i, building_type in enumerate(building_types, start=1):
    subset = emissions_df[emissions_df['Building Type'] == building_type]
    fig.add_trace(go.Scatter(
        x=subset[subset['Emissions Type'] == 'Fossil Emissions']['Year'],
        y=subset[subset['Emissions Type'] == 'Fossil Emissions']['Emissions'],
        fill='tonexty',
        name=f'Fossil fuel CO2 Emissions',
        stackgroup = '1',
        line=dict(color="grey"),
        showlegend = True if building_type == 'OPEN' else False 
    ), row=1, col=i)
    fig.add_trace(go.Scatter(
        x=subset[subset['Emissions Type'] == 'Electricity Emissions']['Year'],
        y=subset[subset['Emissions Type'] == 'Electricity Emissions']['Emissions'],
        fill='tonexty',
        stackgroup = '1',
        name=f'Electricity CO2 Emissions', 
        line=dict(color="red"),
        showlegend = True if building_type == 'OPEN' else False 
    ), row=1, col=i)

    # fig.add_trace(go.Scatter(
    #     x=subset[subset['Emissions Type'] == 'Modified CO2 Emissions']['Year'],
    #     y=subset[subset['Emissions Type'] == 'Modified CO2 Emissions']['Emissions'],
    #     fill='tonexty',
    #     name=f'{building_type} - Modified CO2 Emissions'
    # ), row=1, col=i)
    fig.add_trace(go.Scatter(
        x=subset[subset['Emissions Type'] == 'Solar CO2 Emissions']['Year'],
        y=subset[subset['Emissions Type'] == 'Solar CO2 Emissions']['Emissions'],
        fill='tonexty',
        name=f'Solar CO2 Emissions', 
        line=dict(color="yellow"),
        showlegend = True if building_type == 'OPEN' else False 
    ), row=1, col=i)
    fig.add_trace(go.Scatter(
        x=subset[subset['Emissions Type'] == 'Net Emissions']['Year'],
        y=subset[subset['Emissions Type'] == 'Net Emissions']['Emissions'],
        fill='tonexty',
        text = 'Net Emissions',
        name=f'Net CO2 Emissions', 
        line=dict(color="blue"),
        showlegend = True if building_type == 'OPEN' else False 
    ), row=2, col=i)
    
    fig.add_trace(go.Histogram(x = testcase2[testcase2.building_type == building_type].total_investment_cost, showlegend = False), row = 3, col = i)


# Update layout
fig.update_layout(
    title='CO2 Emissions Over Years by Building Type with heat pumps and A labels',
    xaxis_title='Year',
    yaxis_title='CO2 Emissions (kg)',
    showlegend=True,
    height = 1000, width = 1800,

)

# Update x-axes titles for each subplot
for i in range(1, len(building_types) + 1):
    fig.update_xaxes(title_text='Year', row=1, col=i)

# Update y-axis title only for the first subplot
fig.update_yaxes(title_text='Gross CO2 Emissions (kg)', row=1, col=1)
fig.update_yaxes(title_text='Net CO2 Emissions (kg)', row=2, col=1)
fig.update_yaxes(title_text='Count', row=3, col=1)
fig.update_yaxes(range = [-45000, 120000], row = 1)
fig.update_yaxes(range = [-20000, 100000], row = 2)
fig.update_yaxes(range = [0, 30], row = 3)
for i in range(len(building_types)+ 1):
    fig.update_xaxes(title_text='Investment cost [€]', row=3, col=i)
# Show the plot
fig.show()

In [36]:
testcase2[testcase2.oil_consumption > 0][['heating_consumption', 'domestic_hot_water_consumption', 'electricity_consumption', 'gas_consumption', 'oil_consumption', 'scenario_name']]

,heating_consumption,domestic_hot_water_consumption,electricity_consumption,gas_consumption,oil_consumption,scenario_name


In [37]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# Filter the DataFrame
testcase2 = df[(df['epc_ind'] <= 100) & (df['heating_system'] == 'Gasboiler')& (df['dhw_system'] == 'Gasboiler')].sort_values(by='tco', ascending=True).drop_duplicates(subset=['name'], keep='first')

# Extract the years from the columns
years = sorted([int(col.split('_')[-1]) for col in testcase2.columns if 'emissions_' in col])

# Initialize lists to store the data
data = []

# Collect data for each year and each building type
for year in years:
    for building_type, group in testcase2.groupby('building_type'):
        modified_emissions = group[f'modified_emissions_{year}'].sum()
        fossil_emissions = group[f'fossil_emissions_{year}'].sum()
        electricity_emissions = group[f'electricity_emissions_{year}'].sum()
        solar_emissions = group[f'solar_injection_emissions_{year}'].sum() * -1  # Make solar emissions negative
        net_emissions = group[f'emissions_{year}'].sum()
        data.append([year, building_type, 'Modified CO2 Emissions', modified_emissions])
        data.append([year, building_type, 'Solar CO2 Emissions', solar_emissions])
        data.append([year, building_type, 'Net Emissions', net_emissions])
        data.append([year, building_type, 'Fossil Emissions', fossil_emissions])
        data.append([year, building_type, 'Electricity Emissions', electricity_emissions])

# Create a DataFrame from the collected data
emissions_df = pd.DataFrame(data, columns=['Year', 'Building Type', 'Emissions Type', 'Emissions'])

# Get unique building types
building_types = emissions_df['Building Type'].unique()

# Create subplots
fig = make_subplots(rows=3, cols=len(building_types), shared_yaxes=True, subplot_titles=building_types)

# Add traces for each building type
for i, building_type in enumerate(building_types, start=1):
    subset = emissions_df[emissions_df['Building Type'] == building_type]
    fig.add_trace(go.Scatter(
        x=subset[subset['Emissions Type'] == 'Fossil Emissions']['Year'],
        y=subset[subset['Emissions Type'] == 'Fossil Emissions']['Emissions'],
        fill='tonexty',
        name=f'Fossil fuel CO2 Emissions',
        stackgroup = '1',
        line=dict(color="grey"),
        showlegend = True if building_type == 'OPEN' else False 
    ), row=1, col=i)
    fig.add_trace(go.Scatter(
        x=subset[subset['Emissions Type'] == 'Electricity Emissions']['Year'],
        y=subset[subset['Emissions Type'] == 'Electricity Emissions']['Emissions'],
        fill='tonexty',
        stackgroup = '1',
        name=f'Electricity CO2 Emissions', 
        line=dict(color="red"),
        showlegend = True if building_type == 'OPEN' else False 
    ), row=1, col=i)

    # fig.add_trace(go.Scatter(
    #     x=subset[subset['Emissions Type'] == 'Modified CO2 Emissions']['Year'],
    #     y=subset[subset['Emissions Type'] == 'Modified CO2 Emissions']['Emissions'],
    #     fill='tonexty',
    #     name=f'{building_type} - Modified CO2 Emissions'
    # ), row=1, col=i)
    fig.add_trace(go.Scatter(
        x=subset[subset['Emissions Type'] == 'Solar CO2 Emissions']['Year'],
        y=subset[subset['Emissions Type'] == 'Solar CO2 Emissions']['Emissions'],
        fill='tonexty',
        name=f'Solar CO2 Emissions', 
        line=dict(color="yellow"),
        showlegend = True if building_type == 'OPEN' else False 
    ), row=1, col=i)
    fig.add_trace(go.Scatter(
        x=subset[subset['Emissions Type'] == 'Net Emissions']['Year'],
        y=subset[subset['Emissions Type'] == 'Net Emissions']['Emissions'],
        fill='tonexty',
        text = 'Net Emissions',
        name=f'Net CO2 Emissions', 
        line=dict(color="blue"),
        showlegend = True if building_type == 'OPEN' else False 
    ), row=2, col=i)
    
    fig.add_trace(go.Histogram(x = testcase2[testcase2.building_type == building_type].total_investment_cost, showlegend = False), row = 3, col = i)


# Update layout
fig.update_layout(
    title='CO2 Emissions Over Years by Building Type with gas boilers and A labels',
    xaxis_title='Year',
    yaxis_title='CO2 Emissions (kg)',
    showlegend=True,
    height = 1000, width = 1800,

)

# Update x-axes titles for each subplot
for i in range(1, len(building_types) + 1):
    fig.update_xaxes(title_text='Year', row=1, col=i)

# Update y-axis title only for the first subplot
fig.update_yaxes(title_text='Gross CO2 Emissions (kg)', row=1, col=1)
fig.update_yaxes(title_text='Net CO2 Emissions (kg)', row=2, col=1)
fig.update_yaxes(title_text='Count', row=3, col=1)
fig.update_yaxes(range = [-45000, 120000], row = 1)
fig.update_yaxes(range = [-20000, 100000], row = 2)
fig.update_yaxes(range = [0, 30], row = 3)
for i in range(len(building_types)+ 1):
    fig.update_xaxes(title_text='Investment cost [€]', row=3, col=i)
# Show the plot
fig.show()

In [38]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# Filter the DataFrame
testcase2 = df[(df['epc_ind'] < 100) & (df['heating_system'] == 'Heat Pump') & (df.scenario_name.str.contains('PV full roof')) & (~df.scenario_name.str.contains('PV full roof with battery'))  ].sort_values(by='tco', ascending=False).drop_duplicates(subset=['name'], keep='first')

# Extract the years from the columns
years = sorted([int(col.split('_')[-1]) for col in testcase2.columns if 'emissions_' in col])

# Initialize lists to store the data
data = []

# Collect data for each year and each building type
for year in years:
    for building_type, group in testcase2.groupby('building_type'):
        modified_emissions = group[f'modified_emissions_{year}'].sum()
        solar_emissions = group[f'solar_injection_emissions_{year}'].sum() * -1  # Make solar emissions negative
        data.append([year, building_type, 'Modified CO2 Emissions', modified_emissions])
        data.append([year, building_type, 'Solar CO2 Emissions', solar_emissions])

# Create a DataFrame from the collected data
emissions_df = pd.DataFrame(data, columns=['Year', 'Building Type', 'Emissions Type', 'Emissions'])

# Get unique building types
building_types = emissions_df['Building Type'].unique()

# Create subplots
fig = make_subplots(rows=1, cols=3, shared_yaxes=True, subplot_titles=building_types)

# Add traces for each building type
for i, building_type in enumerate(building_types, start=1):
    subset = emissions_df[emissions_df['Building Type'] == building_type]
    fig.add_trace(go.Scatter(
        x=subset[subset['Emissions Type'] == 'Modified CO2 Emissions']['Year'],
        y=subset[subset['Emissions Type'] == 'Modified CO2 Emissions']['Emissions'],
        fill='tonexty',
        name=f'{building_type} - Modified CO2 Emissions'
    ), row=1, col=i)
    fig.add_trace(go.Scatter(
        x=subset[subset['Emissions Type'] == 'Solar CO2 Emissions']['Year'],
        y=subset[subset['Emissions Type'] == 'Solar CO2 Emissions']['Emissions'],
        fill='tonexty',
        name=f'{building_type} - Solar CO2 Emissions'
    ), row=1, col=i)

# Update layout
fig.update_layout(
    title='CO2 Emissions Over Years by Building Type with heat pump and A labels',
    xaxis_title='Year',
    yaxis_title='CO2 Emissions (kg)',
    showlegend=True
)

# Update x-axes titles for each subplot
for i in range(1, len(building_types) + 1):
    fig.update_xaxes(title_text='Year', row=1, col=i)

# Update y-axis title only for the first subplot
fig.update_yaxes(title_text='CO2 Emissions (kg)', row=1, col=1)

# Show the plot
fig.show()
